# NYC Taxi Fare & Duration
## Model Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import wget
import pickle

sys.path.append("..")

import source.configs as configs
import preprocessing as preprocessing

# Check if we are in COLAB
IN_COLAB = 'google.colab' in sys.modules

In [3]:
if not os.path.exists("../dataset/yellow_tripdata_2022-05.parquet"):
    if IN_COLAB:
        !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-05.parquet "../dataset"
    else:
        wget.download(configs.START_DATASET_URL, "../dataset")

In [4]:
dataset = pd.read_parquet("../dataset/yellow_tripdata_2022-05.parquet")

In [5]:
dataset = preprocessing.add_features(dataset)
dataset = preprocessing.add_targets(dataset)
dataset = preprocessing.fill_na_values(dataset)
dataset = preprocessing.process_outliers(dataset, "delete")

### Features
* trip_distance
* hour_of_day: Extracted from tpep_pickup_datetime
* hour_zone: Extracted from tpep_pickup_datetime. Categorical:[Morning, Noon, Afternoon, Evening, Night]
* rush_hour: Extracted from tpep_pickup_datetime. Binary.
### Targets
* fare_amount
* trip_duration

In [6]:
columns = preprocessing.features + preprocessing.targets
train_dataset = preprocessing.select_features(dataset, columns)
train_dataset, encoders = preprocessing.create_one_hot_encodings(train_dataset, columns)
pickle.dump(encoders, open('encoders.pkl', 'wb'))

In [7]:
train_dataset.columns.to_list()

['trip_distance',
 'hour_of_day',
 'rush_hour',
 'fare_amount',
 'trip_duration',
 'hour_zone_afternoon',
 'hour_zone_evening',
 'hour_zone_morning',
 'hour_zone_night',
 'hour_zone_noon']

In [8]:
for col in train_dataset.isna().sum():
    assert col==0
print("Ok, no NAs in train dataset.")

Ok, no NAs in train dataset.


In [9]:
train_dataset.describe()

,trip_distance,hour_of_day,rush_hour,fare_amount,trip_duration,hour_zone_afternoon,hour_zone_evening,hour_zone_morning,hour_zone_night,hour_zone_noon
count,3.275421e+06,3.275421e+06,3.275421e+06,3.275421e+06,3.275421e+06,3.275421e+06,3.275421e+06,3.275421e+06,3.275421e+06,3.275421e+06
mean,3.243137e+00,1.416682e+01,3.170713e-01,1.369555e+01,1.533393e+01,3.019203e-01,2.459235e-01,2.269525e-01,1.685594e-01,5.664432e-02
std,3.744687e+00,5.748614e+00,4.653355e-01,1.054161e+01,1.091489e+01,4.590909e-01,4.306335e-01,4.188617e-01,3.743624e-01,2.311618e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e+00,1.100000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.140000e+00,1.000000e+01,0.000000e+00,7.000000e+00,7.666667e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.900000e+00,1.500000e+01,0.000000e+00,1.000000e+01,1.241667e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.430000e+00,1.900000e+01,1.000000e+00,1.550000e+01,1.968333e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.030000e+01,2.300000e+01,1.000000e+00,5.550000e+01,6.591667e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
# Split features and targets
X, y = preprocessing.split_dataset(train_dataset)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
y_train_td = y_train["trip_duration"]
model_td = RandomForestRegressor(n_jobs=-1, n_estimators=5)
model_td.fit(X_train_scaled, y_train_td.values.ravel())
predictions = model_td.predict(X_test_scaled)

In [15]:
y_test_td = y_test["trip_duration"]
mse_td = mean_squared_error(y_test_td, predictions)
r2_td = r2_score(y_test_td, predictions)
print(f"MSE: {mse_td}")
print(f"re: {r2_td}")

MSE: 31.668346386910024
re: 0.7345484889010755


In [16]:
y_train_fa = y_train["fare_amount"]
model_fa = RandomForestRegressor(n_jobs=-1, n_estimators=5)
model_fa.fit(X_train_scaled, y_train_fa.values.ravel())
predictions = model_fa.predict(X_test_scaled)

In [17]:
y_test_fa = y_test["fare_amount"]
mse_fa = mean_squared_error(y_test_fa, predictions)
r2_fa = r2_score(y_test_fa, predictions)
print(f"MSE: {mse_fa}")
print(f"re: {r2_fa}")

MSE: 7.469940771627599
re: 0.9328755983533104


In [18]:
pickle.dump(model_fa, open('model_fa.pkl', 'wb'))
pickle.dump(model_td, open('model_td.pkl', 'wb'))
pickle.dump(scaler, open('scaler.pkl', 'wb'))

### Inference

In [19]:
if os.path.exists("model_td.pkl"):
    with open("model_td.pkl", "rb") as model_td_file:
        model_td = pickle.load(model_td_file)
else:
    print("TD model not found!")

if os.path.exists("model_fa.pkl"):
    with open("model_fa.pkl", "rb") as model_fa_file:
        model_fa = pickle.load(model_fa_file)
else:
    print("FA model not found!")

if os.path.exists("scaler.pkl"):
    with open("scaler.pkl", "rb") as scaler_file:
        scaler = pickle.load(scaler_file)
else:
    print("Scaler model not found!")

if os.path.exists("encoders.pkl"):
    with open("encoders.pkl", "rb") as encoders_file:
        encoders = pickle.load(encoders_file)
else:
    print("Encoders model not found!")

Input

In [20]:
pickup_date="2023/12/12"
pickup_time="12:15:12"
tpep_pickup_datetime = pickup_date + " " + pickup_time
trip_distance = "5.5"

In [21]:
data = {
    "trip_distance": [float(trip_distance)], 
    "tpep_pickup_datetime": [pd.to_datetime(tpep_pickup_datetime)] }
df = pd.DataFrame(data)
df = preprocessing.add_features(df)
df.drop(columns="tpep_pickup_datetime", inplace=True)

In [22]:
for encoder_model in encoders:
    encoder, col = encoder_model
    array = pd.DataFrame(df[col].values.reshape(1,-1), columns=[col])
    encoded_data = encoder.transform(array)
    encoded_data = encoded_data.toarray()
    encoded_df = pd.DataFrame(encoded_data, 
                            columns=encoder.get_feature_names_out([col])
    )
    df = pd.concat([df, encoded_df], axis=1, join="inner")
    df.drop(columns=col, inplace=True)

In [23]:
x = scaler.transform(df)

In [24]:
y_fa = round(model_fa.predict(x)[0],2)
y_td = int(model_td.predict(x)[0])

In [25]:
print(f"Trip duration: {y_td} minutes")
print(f"Fare amount: $ {y_fa}")

Trip duration: 27 minutes
Fare amount: $ 20.09
